In [17]:
import cv2
import numpy as np
import time
import random

In [18]:
Game_Duration = 60  # Game duration in seconds
Start_Time = time.time() # Records when the game starts
Score = 0
Misses = 0
Decepticon_Lifetime = 5 # Decepticon's life span on screen
Decepticons = {} # To track decepticons on screen + their spawn times


In [19]:
#Random spawn of decepticons
def spawning_decepticons(frame, Decepticons):
    if len(Decepticons) < 5:  # Limit the number of decepticons on screen
        if random.random() < 0.02:  # 2% chance to spawn a new decepticon each frame
            x = random.randint(0, frame.shape[1] - 50)
            y = random.randint(0, frame.shape[0] - 50)
            Decepticons[(x, y)] = time.time()  # Store spawn time
            cv2.rectangle(frame, (x, y), (x + 50, y + 50), (0, 0, 255), -1)  # Draw decepticon
    return frame, Decepticons

In [20]:
# Remove decepticons after finishing their life span + add to misses
def remove_Decepticons(Decepticons):
    global Misses
    current_time = time.time() #gets the current time
    to_remove = [pos for pos, spawn_time in Decepticons.items() if current_time - spawn_time > Decepticon_Lifetime] #List comprehension iterates over Decepticons, it calculates how long the decepticon has been on screen, if it exceeds the life span it is marked for removal, it results in a list containing the decepticons that overstayed
    for pos in to_remove:
        del Decepticons[pos] #removes the decepticons that overstayed
        Misses += 1 #increments misses by 1 every time a decepticon is removed
    return Decepticons

In [21]:
#To draw decepticons on the frame
#DECEPTICON_SIZE = 50
#def draw_decepticons(frame, decepticons):
    #for (x, y), spawn_time in decepticons.items():
        #cv2.rectangle(frame, (x, y), (x + DECEPTICON_SIZE, y + DECEPTICON_SIZE), (0, 0, 255), -1)

In [22]:
def score_and_misses(frame):
    global Score, Misses
    cv2.puttext(frame, f"Score:{Score}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2) # Displays score at the top left corner in green
    cv2.puttext(frame, f"Misses:{Misses}", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2) # Displays misses at the top left corner in red
    # cv2.puttext(image, text, position, font, font_scale, color, thickness)
    return frame

In [ ]:
def main_gameloop():
    global Score, Misses, Decepticons
    # opens webcam for video capture
    cap = cv2.VideoCapture(0)
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Failed to capture frame.")
        break
        elapsed_time = time.time() - Start_Time #Calculates how much time has passed since the game started
        Time_Left = Game_Duration - elapsed_time # calculates how much time is left
        if Time_Left <= 0:
            cv2.putText(frame, "GAME OVER", (200, 200), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 3) # Displays that the game is over
            cv2.putText(frame, f"Final Score: {Score}", (200, 250), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2) # Displays the final score
            cv2.imshow("Wack-a-Decepticon", frame)
            cv2.waitKey(5000)  
            break  

        Decepticons = spawning_decepticons(frame, Decepticons) # calls that function that spawns decepticons
        
        Decepticons = remove_decepticons(Decepticons) # calls the function that removes decepticons after their life span


        #draw_decepticons(frame, decepticons) # draws the decepticons on the frame

        
        display_score_and_misses(frame, score, misses, time_left) # displays the score and misses on the frame

        
        cv2.imshow("Wack-a-Decepticon", frame)

        
        key = cv2.waitKey(30) & 0xFF # waits for 30 ms for a key press
        if key == 27: # if the key pressed is 'ESC'
            break  

    
    cap.release()
    cv2.destroyAllWindows()

